Измалкова Дарья БКЛ182

# Домашнее задание №1

In [13]:
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymystem3 import Mystem
import requests
from bs4 import BeautifulSoup
from collections import Counter

russian_stopwords = stopwords.words("russian")
stops = set(stopwords.words('russian'))
m = Mystem()

#### Функция, которая приводит текст к нижнему регистру и лемматизирует его. 

In [2]:
def token_and_lemm(text):
    lemmas = ''.join(m.lemmatize(text))
    clean_text = [w.lower() for w in word_tokenize(lemmas) if w.isalpha() and w not in stops]
    return clean_text

#### Функция, возвращающая список не супер редких слов из списка текстов

In [3]:
def get_indicators(texts):
    indics = set()
    word_count = Counter()
    for text in texts:
        word_count = word_count + Counter(token_and_lemm(text))
        

    for key, value in word_count.items():
        if value  > 2:
            indics.add(key)
    return indics

#### Функция, которая находит отличительные слова

In [4]:
def segregate(pos, neg):
    pos_count = get_indicators(pos)
    neg_count = get_indicators(neg)

    pos_words = pos_count - neg_count
    neg_words = neg_count - pos_count
    return pos_words, neg_words

#### Функция, которая оценивает текст 

In [5]:
def assess(pos_ind, neg_ind, text):
    count = Counter()
    for word in token_and_lemm(text):
        if word in pos_ind:
            count['pos'] += 1
        elif word in neg_ind:
            count['neg'] += 1

    if count['pos'] > count['neg']:
        
        return 'positive'
    elif count['pos'] < count['neg']:
        return 'negative'
    else:
        return 'undecided'

### Сбор данных: 
взяла отзывы о фильме "Довод" с www.afisha.ru

In [6]:
def make_soup(page):
    html = requests.get(page)
    soup = BeautifulSoup(html.text,'html.parser')
    return soup

In [7]:
page = 'https://www.afisha.ru/movie/258600/reviews/'

In [8]:
def get_data(page):
    data = []
    
    page_count = 1
    adds = ['positive/','negative/']
    
    for add in adds:
        count = 0
        
        if 'positive' in add:
                value = 'positive'
        elif 'negative' in  add:
                value = 'negative'
                
        soup = make_soup(page + add)       
        for r in soup.find_all(itemprop = 'review'):
            review = str(r.find(itemprop = 'reviewBody'))
# .text / .get_text() почему-то выдавали пустоту
    
            count += 1
            
            if count == 25:
                add2 = add + 'page2/'
                soup = make_soup(page + add2)       
                for r in soup.find_all(itemprop = 'review'):
                    review = str(r.find(itemprop = 'reviewBody'))
                    data.append({'text' : review, 'rating' : value})

            data.append({'text' : review, 'rating' : value})
    return data

In [9]:
data = get_data(page)

#### Делю отзывы на список позитивных и негативных и отбираю 10 на тест.

In [16]:
test_data = []
pos_data = []
neg_data = []
for item in data:
    
    if item['rating'] == 'positive':
        if len(pos_data) < 45:
            pos_data.append(item['text'])
        else:
                test_data.append(item)
    elif item['rating'] == 'negative':
            if len(neg_data) < 45:
                neg_data.append(item['text'])
            else:
                test_data.append(item)
        

#### Собираю списки отличительных слов.

In [17]:
pos_ind, neg_ind = segregate(pos_data,neg_data)

### Проверка.

In [18]:
predictions = []
reality = []
for item in test_data: 
    predictions.append(assess(pos_ind, neg_ind, item['text']))
    reality.append(item['rating'])

print("Accuracy: %.4f" % accuracy_score(predictions, reality))

Accuracy: 0.7000


#### Как улучшить программу? 
<br>

 <li> 1) Собрать выборку побольше </li>
 <li> 2) Попробовать с нейросетью, чтобы не полагаться на конкретные тексты и вхождения в них</li>
 <li> 3) Возможно есть способ оценить положительность/отрицательность не только на лексике</li>